<a href="https://colab.research.google.com/github/thingumajig/colab-experiments/blob/master/predict_BERT_BiLSTM_CRF_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## clone from git

In [0]:
#!pip install --upgrade tensorflow-gpu
!pip install seqeval
!pip install bert-tensorflow

!pip list | grep tensorflow
#!ps ax | grep python


  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
     |████████████████████████████████| 71kB 6.0MB/s 
bert-tensorflow          1.0.1                
mesh-tensorflow          0.0.5                
tensorflow               1.14.0               
tensorflow-estimator     1.14.0               
tensorflow-hub           0.5.0                
tensorflow-metadata      0.14.0               
tensorflow-probability   0.7.0                


In [80]:

import sys

# bert_path = 'bert'
# !test -d $bert_path || git clone https://github.com/google-research/bert $bert_path
# if not bert_path in sys.path:
#   sys.path.insert(0, bert_path)



bert_ner_path = 'bert-ner'  
!test -d $bert_ner_path || git clone https://github.com/thingumajig/colab-experiments $bert_ner_path

if not bert_ner_path in sys.path:
  sys.path.insert(0, bert_ner_path)

#!cp /content/BERT-NER/metrics.py /content/metrics.py



fatal: not a git repository (or any of the parent directories): .git


In [82]:
#!rm -rfv $bert_ner_path/bert
!git -C $bert_ner_path pull 

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 3), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/thingumajig/colab-experiments
   a80c031..b407211  master     -> origin/master
Updating a80c031..b407211
Fast-forward
 bert_bilstm_model.py |  20 +++---
 render_results.py    | 167 +++++++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 177 insertions(+), 10 deletions(-)
 create mode 100644 render_results.py


##Model

In [0]:
#@title Select model:
import os

bert_model = "https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip" #@param ["https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip", "https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip"]
bert_model_zip = os.path.basename(bert_model)
bert_model_dir = os.path.splitext(bert_model_zip)[0]

!test -d $bert_model_dir || wget $bert_model


!test -d $bert_model_dir || unzip /content/$bert_model_zip

--2019-07-25 17:13:59--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 2a00:1450:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1242589256 (1.2G) [application/zip]
Saving to: ‘wwm_cased_L-24_H-1024_A-16.zip’

wwm_cased_L-24_H-10 100%[===================>]   1.16G  64.8MB/s    in 21s     

2019-07-25 17:14:20 (57.2 MB/s) - ‘wwm_cased_L-24_H-1024_A-16.zip’ saved [1242589256/1242589256]

Archive:  /content/wwm_cased_L-24_H-1024_A-16.zip
   creating: wwm_cased_L-24_H-1024_A-16/
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_cased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_cased_L-24_H-1

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##init flags

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import json
from bert import modeling
from bert import optimization
from bert import tokenization
import tensorflow as tf
import codecs
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.contrib import crf
from tensorflow.contrib import rnn, cudnn_rnn

from tensorflow.contrib.tpu import TPUEstimator, TPUConfig
from tensorflow.python.ops import math_ops

import tf_metrics
import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

flags = tf.flags

FLAGS = flags.FLAGS



# print(FLAGS.__dict__['__wrapped'])
def del_all_flags(FLAGS, r = []):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        if keys in r:
          FLAGS.__delattr__(keys)
 
# allows rerun cell
del_all_flags(FLAGS, r = ['bert_config_file','data_dir',
                         "task_name", "output_dir","init_checkpoint",
                         "do_lower_case", "do_train", "use_tpu", "do_predict",
                         "use_crf", "train_batch_size", "eval_batch_size",
                         "learning_rate", "num_train_epochs", "warmup_proportion",
                         "bert_dropout_rate", "bilstm_dropout_rate",
                          "save_checkpoints_steps", "save_summary_steps",
                         'data_config_path', "num_tpu_cores", "master",
                          "vocab_file", "keep_checkpoint_max",                          
                          'lstm_size', 'max_seq_length',
                          'predict_batch_size', 'iterations_per_loop',
                          
                         ])
train_model_path = '/content/drive/My Drive/ner_checkpoints'
if not os.path.exists(train_model_path):
    os.makedirs(train_model_path)

flags.DEFINE_string("data_dir", f'/content/{bert_ner_path}/CoNLL/data', "The input datadir. ex) 'NERdata'")
flags.DEFINE_string("bert_config_file", os.path.join('/content',bert_model_dir,'bert_config.json'), "The config json file corresponding to the pre-trained BERT model. ex) 'bert_config.json'")

flags.DEFINE_string("task_name", 'NER', "The name of the task to train. ex) 'NER'")

flags.DEFINE_string("output_dir", train_model_path, "The output directory where the model checkpoints will be written. ex) 'output'")

flags.DEFINE_string("init_checkpoint", f"/content/{bert_model_dir}/bert_model.ckpt", "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool("do_lower_case", False, "Whether to lower case the input text.")

flags.DEFINE_integer("max_seq_length", 128, "The maximum total input sequence length after WordPiece tokenization.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")
flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

flags.DEFINE_bool("use_crf", True, "Whether to use CRF decoding.")

flags.DEFINE_integer("train_batch_size", 16, "Total batch size for training.")
flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")
flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")
flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0, "Total number of training epochs to perform.")
flags.DEFINE_float("warmup_proportion", 0.1, "Proportion of training to perform linear learning rate warmup for. E.g., 0.1 = 10% of training.")
flags.DEFINE_float("bert_dropout_rate", 0.2, "Proportion of dropout for bert embedding.")
flags.DEFINE_float("bilstm_dropout_rate", 0.2, "Proportion of dropout for bilstm.")
flags.DEFINE_integer("save_checkpoints_steps", 1000, "How often to save the model checkpoint.")
flags.DEFINE_integer("save_summary_steps", 100, "Save summaries every this many steps")
flags.DEFINE_integer("keep_checkpoint_max", 20, "The maximum number of recent checkpoint files to keep")
flags.DEFINE_integer("iterations_per_loop", 1000, "How many steps to make in each estimator call.")

flags.DEFINE_string("vocab_file", f'/content/{bert_model_dir}/vocab.txt', "The vocabulary file that the BERT model was trained on. ex) 'vocab.txt'")

flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_integer( "num_tpu_cores", 8, "Only used if `use_tpu` is True. Total number of TPU cores to use.")

flags.DEFINE_string('data_config_path', 'data.conf', 'data config file, which save train and dev config')

flags.DEFINE_integer('lstm_size', 128, 'size of lstm units')

print(FLAGS.__dict__['__wrapped'])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:
  --bert_config_file: The config json file corresponding to the pre-trained BERT
    model. ex) 'bert_config.json'
    (default: '/content/wwm_cased_L-24_H-1024_A-16/bert_config.json')
  --bert_dropout_rate: Proportion of dropout for bert embedding.
    (default: '0.2')
    (a number)
  --bilstm_dropout_rate: Proportion of dropout for bilstm.
    (default: '0.2')
    (a number)
  --data_config_path: data config file, which save train and dev config
    (default: 'data.conf')
  --data_dir: The input datadir. ex) 'NERdata'
    (default: '/content/bert-ner/CoNLL/data')
  --[no]do_lower_case: Whether to lower case the input text.
    (default: 'false')
  --[no]do_predict: Whether to run the model in inference mode on the test set.
    (default: 'true')
  --[no]do_train: Whether to run training.
    (default: 'false')
  --eval_batch_size: Total batch size for eval.
    (default: '8')
    (an integer)
  --init_checkpoint: Initial

## main

In [0]:
import bert_bilstm_model as m

processors = {
    "ner": m.NerProcessor
}

def create_estimator(label_list):
    bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)
    if FLAGS.max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            (FLAGS.max_seq_length, bert_config.max_position_embeddings))

    tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

    tpu_cluster_resolver = None
    if FLAGS.use_tpu and FLAGS.tpu_name:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=FLAGS.master,
        model_dir=FLAGS.output_dir,
        save_checkpoints_steps=FLAGS.save_checkpoints_steps,
        keep_checkpoint_max=FLAGS.keep_checkpoint_max,
        save_summary_steps=FLAGS.save_summary_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=FLAGS.iterations_per_loop,
            num_shards=FLAGS.num_tpu_cores,
            per_host_input_for_training=is_per_host))

    model_fn = m.model_fn_builder(
        bert_config=bert_config,
        num_labels=len(label_list) + 1,  # 1 for '0' padding
        init_checkpoint=FLAGS.init_checkpoint,
        learning_rate=FLAGS.learning_rate,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=FLAGS.use_tpu)

    ws = None
    if os.path.exists(FLAGS.output_dir):
        print(f'================== use WarmStartSettings from {FLAGS.output_dir}')
        ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=FLAGS.output_dir)
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=FLAGS.use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=FLAGS.train_batch_size,
        eval_batch_size=FLAGS.eval_batch_size,
        predict_batch_size=FLAGS.predict_batch_size,
        warm_start_from=ws
    )
    return estimator, tokenizer


def train():
    task_name = FLAGS.task_name.lower()
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    processor = processors[task_name]()

    label_list = processor.get_labels()

    estimator, tokenizer = create_estimator(label_list)

    if os.path.exists(FLAGS.data_config_path):
        with codecs.open(FLAGS.data_config_path) as fd:
            print(f'=========== load existed config:{FLAGS.data_config_path}')
            data_config = json.load(fd)
    else:
        data_config = {}


    train_examples = None
    num_train_steps = None
    num_warmup_steps = None
    if len(data_config) == 0:
        train_examples = processor.get_train_examples(FLAGS.data_dir)
        num_train_steps = int((len(train_examples) / FLAGS.train_batch_size) * FLAGS.num_train_epochs)
        num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
        data_config['num_train_steps'] = num_train_steps
        data_config['num_warmup_steps'] = num_warmup_steps
        data_config['num_train_size'] = len(train_examples)
    else:
        num_train_steps = int(data_config['num_train_steps'])
        num_warmup_steps = int(data_config['num_warmup_steps'])

    # prepare train_input_fn
    if data_config.get('train.tf_record_path', '') == '':
        train_file = os.path.join(FLAGS.output_dir, "train.tf_record")
        m.filed_based_convert_examples_to_features(
            train_examples, label_list, FLAGS.max_seq_length, tokenizer, train_file)
    else:
        train_file = data_config.get('train.tf_record_path')
    print(f'================== train file: {train_file}')
    num_train_size = num_train_size = int(data_config['num_train_size'])
    print("***** Running training *****")
    print(f"  Num examples = {num_train_size} Batch size = {FLAGS.train_batch_size} Num steps = {num_train_steps}")
    train_input_fn = m.file_based_input_fn_builder(
        input_file=train_file,
        seq_length=FLAGS.max_seq_length,
        is_training=True,
        drop_remainder=True)
    # prepare eval_input_fn
    if data_config.get('eval.tf_record_path', '') == '':
        eval_examples = processor.get_dev_examples(FLAGS.data_dir)
        eval_file = os.path.join(FLAGS.output_dir, "eval.tf_record")
        m.filed_based_convert_examples_to_features(
            eval_examples, label_list, FLAGS.max_seq_length, tokenizer, eval_file)
        data_config['eval.tf_record_path'] = eval_file
        data_config['num_eval_size'] = len(eval_examples)
    else:
        eval_file = data_config['eval.tf_record_path']
    num_eval_size = data_config.get('num_eval_size', 0)
    print("***** Running evaluation *****")
    print(f"  Num examples = {num_eval_size} Batch size = {FLAGS.eval_batch_size}")
    eval_steps = None
    if FLAGS.use_tpu:
        eval_steps = int(num_eval_size / FLAGS.eval_batch_size)
    eval_drop_remainder = True if FLAGS.use_tpu else False
    eval_input_fn = m.file_based_input_fn_builder(
        input_file=eval_file,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=eval_drop_remainder)
    # train and evaluate tf.estimator.experimental.
    hook = tf.estimator.experimental.stop_if_no_decrease_hook(
        estimator, 'eval_f', 3000, min_steps=30000, run_every_secs=120)
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps, hooks=[hook])
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, throttle_secs=120)
    tp = tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    result = tp[0]

    output_eval_file = os.path.join(FLAGS.output_dir, "eval_results.txt")
    with codecs.open(output_eval_file, "w", encoding='utf-8') as writer:
        print("***** Eval results *****")
        for key in sorted(result.keys()):
            print(f"  {key} = {str(result[key])}")
            writer.write(f"{key} = {str(result[key])}\n")


    if not os.path.exists(FLAGS.data_config_path):
        with codecs.open(FLAGS.data_config_path, 'a', encoding='utf-8') as fd:
            json.dump(data_config, fd)


def predict(estimator, tokenizer, label_list, text):

    # prepare predict_input_fn
    # predict_examples = processor.get_test_examples(FLAGS.data_dir)
    # predict_examples = processor.get_predict_examples(os.path.join(FLAGS.output_dir, filename))
    predict_examples = processor.get_predict_examples_from_str(text)
    print(f'================= Number of predict examples:{len(predict_examples)}')

    predict_file = os.path.join(FLAGS.output_dir, "predict.tf_record")
    m.filed_based_convert_examples_to_features(predict_examples, label_list,
                                               FLAGS.max_seq_length, tokenizer,
                                               predict_file, mode="test")

    print("***** Running prediction*****")
    print(f"  Num examples = {len(predict_examples)} Batch size = {FLAGS.predict_batch_size}")
    if FLAGS.use_tpu:
        # Warning: According to tpu_estimator.py Prediction on TPU is an
        # experimental feature and hence not supported here
        raise ValueError("Prediction in TPU not supported")
    predict_drop_remainder = True if FLAGS.use_tpu else False
    predict_input_fn = m.file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=predict_drop_remainder)

    # predict
    result = estimator.predict(input_fn=predict_input_fn)
    
    return predict_examples, result 



# run

In [0]:
  task_name = FLAGS.task_name.lower()
  if task_name not in processors:
      raise ValueError("Task not found: %s" % (task_name))
  processor = processors[task_name]()
  label_list = processor.get_labels()


  estimator, tokenizer = create_estimator(label_list)


W0725 19:01:33.328863 140252899514240 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f8ecc864b70>) includes params argument, but params are not passed to Estimator.
W0725 19:01:33.331225 140252899514240 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


================== use WarmStartSettings from /content/drive/My Drive/ner_checkpoints


In [86]:
text = '''
Interlocal agreement for consulting services.

This is an Interlocal Agreement entered into pursuant to the authority of ch. 39.34 RCW, between the Cities of Lacey, Olympia, Tumwater, and Thurston County, hereinafter referred to as the jurisdictions, for the purpose of sharing the collective cost of negotiating with Comcast Cable for the renewal of each entity’s cable franchise.  
All parties agree to the terms outlined herein:


    1. The jurisdictions agree that it would be mutually beneficial to negotiate with Comcast Cable jointly rather than individually for the purposes of renewing each entity’s cable franchise.  By doing so, the cost of negotiating with Comcast Cable will be incrementally reduced for each party.


    2. The total cost, including but not limited to consultant fees and legal review necessary to finalize cable franchise negotiations with Comcast Cable for the parties involved, will be paid by the jurisdictions on a per subscriber basis, according the provisions of Paragraph 3, herein.   


    3. In order to allocate the initial distribution of the costs of negotiations on a per subscriber basis, the jurisdictions agree with the subscriber data outlined in Attachment “A.”   For example, bBased on the current subscriber data in Attachment “A”, Thurston County will pay 51%, Lacey  16%, Olympia  25%, and Tumwater  8% of the total costs for the consultant hired pursuant to Paragraph 4, herein, subject to the termination provisions of Paragraph 11, herein.  
	Attachment “A” will be updated at the beginning of each year with the most current cable subscribers.  
	This update will represent an amendment to this Agreement, pursuant to Paragraph 10, herein. 
	Each such update (amendment) will act to supersede the prior Attachment A, in order to continue the distribution of the cost of franchise renewal on a per subscriber basis.


4.	The jurisdictions will jointly produce a Request for Proposals (RFP), to include a Scope of Work, for purposes of hiring a consultant (or a consulting firm) to negotiate with Comcast Cable on behalf of the jurisdictions. 
A draft Scope of Work is attached to this Agreement, as Attachment B. 
The Scope of Work will be divided into two phases: (1) Review and Recommendation; and (2) Negotiation and Final Franchise Agreement. 
The draft Scope of Work is subject to change, depending upon the negotiation process with the consultant (or firm) that is hired. 
The final Scope of Work will be incorporated into the professional services agreement with the consultant that is hired through the RFP process. 
All jurisdictions must agree to the final Scope of Work, by letter indicating such agreement, addressed to John Tennis, Public Information Program Manager, Thurston County, prior to award of the consultant contract by Thurston County.
 
5.	The term of this Agreement shall be effective upon the approval of each party’s governing body, and recording of the Agreement with the Thurston County Auditor.  
This Agreement shall terminate on June 30, 2007 unless extended by agreement of the parties.    

    6. Thurston County will act as the Lead Agency on behalf of the jurisdictions.  
	The Lead Agency, for the purpose of this Agreement, will award the contract for cable consultant as well as collect payments under this Agreement and administer payments to satisfy the consultant contract.  
	Thurston County rules and procedures will be followed for the purpose of awarding the contract.  
	[The last sentence in this paragraph doesn’t make sense. 
	If Thurston County is going to be the lead agency and award the contract, then each jurisdiction’s governing body doesn’t need to execute the contract with the consultant, as this Agreement provides the basis for one jurisdiction to award the contract.]

Thurston County Comment:  In order for the County to agree to execute the consultant contract on behalf of the Jurisdictions, the following language will need to replace the currently proposed section 6 language:

Thurston County will act as the Lead Agency on behalf of the jurisdictions.  
The Lead Agency, for the purpose of this Agreement, will award and execute the contract for cable consultant, collect payments under this Agreement, and administer the consultant contract.  
Thurston County procedures will be followed for the purpose of awarding and executing the consultant contract.  
It is understood by the parties to this Agreement that Thurston County is executing and administering the cable consultant contract on behalf of the jurisdictions and each jurisdiction shall be responsible under the cable consultant contract as if each jurisdiction individually executed said contract.  
The responsibility of the jurisdictions under this section shall survive the termination of this Agreement whether termination is by all parties, or by one or more parties.   

    7.  The parties agree that each of the parties shall, at all times, be solely responsible for the acts or the failure to act of its personnel that occur or arise in any way out of the performance of this Agreement by its personnel only, and to save and hold the other parties and their personnel and officials harmless from all costs, expenses, losses and damages, including cost of defense, incurred as a result of any acts or omissions of the party’s personnel relating to the performance of this Agreement. 
	The provisions of this section shall survive termination of this Agreement, whether termination is by all parties, or by one or more parties.

    8. The parties shall not jointly acquire property and therefore there is no need to set forth a means of disposition of such property.

    9. No separate legal entity is created by this Agreement. 
	No administrator or joint board is created by this Agreement; however, Thurston County will be the lead agency for the purposes set forth in Paragraph 6, herein.

    10. This Agreement may only be modified by mutual agreement of all parties hereto, executed in writing, in the same manner as this Agreement.

    11. This Agreement may be terminated as to any single party, when the party provides notice to all other parties in writing at least 60 days prior to its intended withdrawal from this Agreement.  
	Upon the effective date of termination by a party, the distribution of costs shall be reallocated between the non-terminating parties as set forth in Paragraphs 2 and 3, herein, and will constitute an amendment to this Agreement pursuant to the terms of Paragraph 10, herein. 
	Any and all costs incurred or encumbered by the parties prior to the effective date of termination of a party, will remain the responsibility of the withdrawing party.

Thurston County Comment:  By adding the highlighted language to section 11, this would mean that all jurisdictions would need to mutually agree to the termination.  
Is this what is contemplated by the jurisdictions?  If not, then this language should not be included.

    12. Notice provided for in this Agreement shall be sent by certified mail to the addresses as set forth below. 
	Notice will be deemed received three business days following posting by the U.S. Postmaster.



City of Tumwater, c/o City Administrator, 555 Israel Road SW, Tumwater, WA  98501.

City of Olympia [Add name and address here.]

City of Lacey [Add name and address here.]

Thurston County [Add name and address here.]



    13. This Agreement has been and shall be construed as having been made and delivered within the State of Washington, and it is agreed by each party hereto that this Agreement shall be governed by laws of the State of Washington, both as to interpretation and performance. 
	Any action of law, suit in equity, or judicial proceeding for the enforcement of this Agreement or any provisions thereof, shall be instituted and maintained only in Superior Court in Thurston County, Washington.

    14. If, for any reason, any part, term or provision of this Agreement is held by a court of competent jurisdiction to be illegal, void or unenforceable, the validity of the remaining provisions shall not be affected, and the rights and obligations of the parties shall be construed and enforced as if the Agreement did not contain the particular provision held to be invalid.

    15. The parties agree that this Agreement is the complete expression of the terms hereto and any oral representations or understandings not incorporated herein are excluded. It is also agreed by the parties that the forgiveness of the nonperformance of any provision of this Agreement does not constitute a waiver of the provisions of this Agreement. 

IN WITNESS WHEREOF the parties hereto have caused this Agreement to be executed according to the terms written above.





City of Lacey:						City of Olympia:

By:							By:					

Its:							Its:					

Date:							Date:					

Attest:							Attest:

By:							By:					

Approved as to form:					Approved as to form:

By:							By:					

City of Tumwater:					Thurston County:

							:					
Ralph C. Osgood, Mayor				Chairman
												
							Vice-Chairman
							______________________________
							Commissioner
Date:							Date:					

Attest:							Attest:

By:							By:					

Approved as to form:					Approved as to form:
							By:					
Christy A. Todd, City Attorney


'''

text = '''
Republicans in the Senate have twice in 24 hours blocked the advancement of bills aimed at strengthening election security just hours after former special counsel Robert Mueller warned of the continued threat that foreign powers interfering in US elections.

Senate Majority Leader Mitch McConnell came to the Senate floor Thursday to personally object to House-passed legislation backed by Democrats. This comes after Republican Sen. Cindy Hyde-Smith of Mississippi objected to a trio of bills on Wednesday, in keeping with long standing GOP arguments that Congress has already responded to election security needs for the upcoming election.
Democratic Sens. Mark Warner of Virginia, Richard Blumenthal of Connecticut and Ron Wyden of Oregon had advocated for the bills on the Senate floor, asking for unanimous consent to pass the package, but that ask can be halted with an objection from any senator.
Two of those bills would require campaigns to report to federal authorities any attempts by foreign entities to interfere in US elections, and the third is aimed at protecting from hackers the personal accounts and devices of senators and some staffers.
On Thursday, McConnell called the House-passed legislation Democrats wanted to clear "so partisan it received just one Republican vote over in the House." He added that any election security legislation the Senate takes up must be bipartisan and also alleged that this bill is being pushed by the same Democrats who pushed the "conspiracy theory" of President Donald Trump and Russia.
'''

with codecs.open(os.path.join(FLAGS.output_dir, 'label2id.pkl'), 'rb') as rf:
  label2id = pickle.load(rf)
  id2label = {value: key for key, value in label2id.items()}

for (id, lbl) in id2label.items():
  print(f'{id}={lbl}')

predict_examples, result = predict(estimator, tokenizer, label_list, text)
result = list(result)



1=O
2=B-PER
3=I-PER
4=B-ORG
5=I-ORG
6=B-LOC
7=I-LOC
8=B-MISC
9=I-MISC
10=X
================= Number of predict examples:260
Writing example 0 of 260
*** Example ***
guid: predict-0
tokens: Republicans
input_ids: 101 11115 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [87]:
from render_results import render_predictions_html

#    output_predict_file = os.path.join(FLAGS.output_dir, "pred.txt")
#     write_predictions(predict_examples, result, output_predict_file, id2label)
from IPython.display import HTML, display
display(HTML(
  render_predictions_html(predict_examples, result, id2label)
))


In [88]:
from render_results import render_predictions
render_predictions(predict_examples, result, id2label)

Republicans<B-MISC> in the Senate<B-ORG> have twice in 24 hours blocked the advancement of bills aimed at strengthening election security just hours after former special counsel Robert<B-PER> Mueller<B-PER> warned of the continued threat that foreign powers interfering in US<B-LOC> elections .
Senate<B-ORG> Majority Leader Mitch<B-PER> McConnell<B-PER> came to the Senate<B-ORG> floor Thursday to personally object to House-passed legislation backed by Democrats<B-MISC> .
This comes after Republican<B-MISC> Sen.<B-ORG> Cindy<B-PER> Hyde-Smith<B-PER> of Mississippi<B-LOC> objected to a trio of bills on Wednesday , in keeping with long standing GOP arguments that Congress<B-ORG> has already responded to election security needs for the upcoming election .
Democratic<B-MISC> Sens .
Mark<B-PER> Warner<B-PER> of Virginia<B-PER> , Richard<B-PER> Blumenthal<B-ORG> of Connecticut<B-LOC> and Ron<B-PER> Wyden<B-PER> of Oregon<B-LOC> had advocated for the bills on the Senate<B-ORG> floor , asking fo

In [0]:
#print(FLAGS.__dict__['__wrapped'])
flags.DEFINE_string("f", 'f', "f")
FLAGS.data_dir

'/content/bert-ner/CoNLL/data'

In [0]:
!ps ax | grep python

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize
#my_sent = "Hi man, how have you been?"
#print(word_tokenize(my_sent))

ls = sent_tokenize(text)
for (i,l) in enumerate(ls): 
  print(f'====== line: {i} ============')
  print(l)



#!zip model.ckpt-2632.zip "/content/drive/My Drive/ner_checkpoints/model.ckpt-2000.data-00000-of-00001" "/content/drive/My Drive/ner_checkpoints/model.ckpt-2632.index" "/content/drive/My Drive/ner_checkpoints/model.ckpt-2632.meta" "/content/drive/My Drive/ner_checkpoints/label2id.pkl"

     28 ?        Sl     0:03 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
    133 ?        Ssl    0:07 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-317cbf9d-4c91-465b-bf56-800c930e6f59.json
    422 ?        S      0:00 /usr/bin/python3 -Wignore:::pip._internal.cli.base_command -c from multiprocessing.semaphore_tracker import main;main(58)
    546 ?        S      0:00 /bin/bash -c ps ax | grep python
    548 ?        S      0:00 grep python
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
====== line: 0 ============

INTERLOCAL AGREEMENT FOR CONSULTING SERVICES

This is an Interlocal Agreement entered into pursuant to the authority of ch.
====== line: 1 ============
39.34 RCW, between the Cities of Lacey, Olympia, Tumwater, and Thurston County, hereinafter referred to as the JU

In [69]:
from IPython.display import HTML, display
xxx = """
<style> 
.x1 {
	  padding: .2em .3em;
    padding-top: 0.2em;
    padding-right: 0.3em;
    padding-bottom: 0.2em;
    padding-left: 0.3em;
    margin: 0 .25em;
    margin-top: 0px;
    margin-right: 0.25em;
    margin-bottom: 0px;
    margin-left: 0.25em;
    line-height: 1;
    display: inline-block;
    border-radius: .25em;
    border-top-left-radius: 0.25em;
    border-top-right-radius: 0.25em;
    border-bottom-right-radius: 0.25em;
    border-bottom-left-radius: 0.25em;
}

.x2 {
    box-sizing: border-box;
    content: attr(data-entity);
    font-size: .55em;
    line-height: 1;
    padding: .35em .35em;
    padding-top: 0.35em;
    padding-right: 0.35em;
    padding-bottom: 0.35em;
    padding-left: 0.35em;
    border-radius: .35em;
    text-transform: uppercase;
    display: inline-block;
    vertical-align: middle;
    margin: 0 0 .15rem .5rem;
    margin-top: 0px;
    margin-right: 0px;
    margin-bottom: 0.15rem;
    margin-left: 0.5rem;
    background: #fff;
    background-image: initial;
    background-position-x: initial;
    background-position-y: initial;
    background-size: initial;
    background-repeat-x: initial;
    background-repeat-y: initial;
    background-attachment: initial;
    background-origin: initial;
    background-clip: initial;
    background-color: rgb(255, 255, 255);
    font-weight: 700;
}


</style>

<p>Rounded corners for an element with a specified background color:</p>kljhkhlkj
<mark class="x1" style='background-color: red;'>Rounded corners!<sub class="x2">PERS</sub></mark>
<p>Rounded corners for an element with a border:</p>
<p id="rcorners2">Rounded corners!</p>
<p>Rounded corners for an element with a background image:</p>

"""



display(HTML(xxx), metadata=dict(isolated=True))